In [45]:
# Add more data, now have lat, lon, and area of each cell
# Merge WFP Price data w/ shapefile
# Rasterize the shapefile
# Logistic regression on whether an area experiences a price spike
# Use NDVI and SPI in an area... maybe trade data? macroeconomic indicators? crop calendars?

# Add a masked layer using Logistics Cluster Global Obstacles data
import os
import pickle
import pandas as pd
pd.options.display.max_rows = 1000
from hdx.hdx_configuration import Configuration
from hdx.data.dataset import Dataset
Configuration.create(hdx_site="test", hdx_read_only=True)

ConfigurationError: Configuration already created!

In [3]:
### Load data from Humanitarian Data Exchange ###

print("Started")

from hdx.hdx_configuration import Configuration
from hdx.data.dataset import Dataset
#Configuration.create(hdx_site="test", hdx_read_only=True)

datasets = Dataset.search_in_hdx('Food and Commodity WFP', rows=10)
resource = Dataset.get_resources(datasets[0])
url, path = resource[0].download()
print("Data path retrieved")

[{'cache_last_updated': None, 'package_id': '4fdcd4dc-5c2f-43af-a1e4-93c9b6539a27', 'datastore_active': True, 'id': 'bd88a565-bf6f-4827-b07b-fb3a65bbb01a', 'size': None, 'revision_last_updated': '2017-07-26T17:02:03.635958', 'state': 'active', 'hash': '', 'description': 'Flat CSV file with > 500k records.', 'format': 'CSV', 'hdx_rel_url': 'http://vam.wfp.org/sites/data/WFPVAM_FoodPrices_24-7-2017.csv', 'last_modified': None, 'url_type': 'api', 'originalHash': '97196323', 'mimetype': None, 'cache_url': None, 'name': 'WFPVAM_FoodPrices.csv', 'created': '2017-07-26T17:01:52.600308', 'url': 'http://vam.wfp.org/sites/data/WFPVAM_FoodPrices_24-7-2017.csv', 'mimetype_inner': None, 'position': 0, 'revision_id': '4bcf2738-8fe5-4115-b182-cd4911519bee', 'resource_type': 'api'}]


KeyboardInterrupt: 

In [ ]:
### Load data from pickle ###
### SEE BELOW ###

In [ ]:
import pandas as pd
pd.options.display.max_columns = 1000
pd.options.display.max_rows = 100

wfp_food_price_data = pd.read_csv(path,encoding = "ISO-8859-1")
df = wfp_food_price_data

print("Data loaded")

mkt_names = df.loc[:,"mkt_name"].unique()
print(mkt_names)
commodities = {}
for mkt in mkt_names:
    commodities[mkt] = df.loc[df.loc[:,"mkt_name"]==mkt, "cm_name"].unique()

print("Markets and Commodities Set")
    
from datetime import date
from sklearn import linear_model
from sklearn.metrics import mean_squared_error, r2_score
import numpy as np

# can be used for the toordinal function
def create_ordinal_date_column(date_tuple):
    return(date.toordinal(date(*date_tuple)))

def add_dummy_columns(df, src_col):
    """Create additional columns on df as dummy variables 
        for values in src_col (i.e. months) """
    
    dummy_data = df.loc[:,(src_col)]
    dummy_vals = dummy_data.unique()
    
    for val in dummy_vals:
        dummy = (dummy_data == val).astype(int)
        df.loc[:,("month_"+str(val))] = dummy
    
    return(df)

def sanity_check(df):
    # TO DO: check whether there are at least 3 records for each month
    months = df["mp_month"]
    #print(months)
    ct_months = np.zeros(12)
    for month in months:
        ct_months[month-1] +=1
        
    #print(ct_months)
    for ct in ct_months:
        if ct < 3:
            return(False)
    return(True)

def calc_alps(dev):
    if(dev < .25):
        return("white")
    elif(dev < 1):
        return("yellow")
    elif(dev < 2):
        return("orange")
    else:
        return("red")

for mkt in mkt_names:
    for cmdty in commodities[mkt]:
        print("Market:", mkt, ", Commodity:", cmdty)
        selection = (df["mkt_name"]==mkt) & (df["cm_name"]==cmdty)
        
        price_history = df.loc[(selection), :]
        
        # Sanity check - at least 3 years of data for each month?
        if(not sanity_check(price_history)):
            print("Not enough raw training data")
            continue
        
        # Create ordinal date column
        date_nums = list(zip(price_history["mp_year"], price_history["mp_month"], np.ones(price_history.shape[0]).astype(int)))
        ordinal_dates = list(map(create_ordinal_date_column, date_nums))
        price_history.loc[:,("ordinal_dates")] = ordinal_dates
        
        # Create dummy columns for each month
        price_history = add_dummy_columns(price_history, "mp_month")
        
        # Create training and label data
        training_cols = ["ordinal_dates", "month_1", "month_2", "month_3",
                        "month_4", "month_5", "month_6",
                        "month_7", "month_8", "month_9",
                        "month_10", "month_11", "month_12"]
        X = price_history.loc[:, training_cols]
        Y = price_history.loc[:,"mp_price"]
        
        lm = linear_model.LinearRegression()
        lm.fit(X, Y)

        #print(lm.coef_)
        #print(lm.intercept_)
        
        # Calculate model residuals
        Y_hat = lm.predict(X)
        residuals = Y - Y_hat
        #print(residuals)
        
        # Divide by standard deviation of residuals
        resid_std_dev = np.sqrt(mean_squared_error(Y, Y_hat))
        ## ^ how to find from lm object?
        std_devs = residuals / resid_std_dev
        
        # Retrain model without first pass outliers
        price_history_tame = price_history.loc[(std_devs > -1) & (std_devs < 1), :]
        
        if(not sanity_check(price_history_tame)):
            print("Not enough tame training data")
            continue
        
        X_tame = price_history.loc[:,training_cols]
        Y_tame = price_history.loc[:,("mp_price")]
                
        lm_tame = linear_model.LinearRegression()
        lm_tame.fit(X_tame, Y_tame)
        
        Y_hat_tame = lm_tame.predict(X)
        residuals_tame = Y - Y_hat_tame
        #print(residuals_tame)
        
        # Divide by standard error of estimatation
        resid_std_dev_tame = np.sqrt(mean_squared_error(Y, Y_hat_tame))
        ## ^ how to find from lm object?
        std_devs_tame = residuals_tame / resid_std_dev_tame
        
        # Calculate ALPS
        ALPS = list(map(calc_alps, std_devs_tame))
        print("setting ALPS on current selection")
        df.loc[selection, "ALPS"] = ALPS
        print(type(list(lm_tame.coef_)))
        df.loc[selection, "Fitted Model"] = lm
        df.loc[selection, "Fitted Model Tame"] = lm_tame
        df.loc[selection, "Model Residual"] = std_devs_tame
        df.loc[selection, "Model Std Error of Residuals"] = resid_std_dev_tame

df.to_pickle("/Users/nathansuberi/Desktop/RW_Data/wfp_alps.pkl")

In [2]:
import pandas as pd
wfp_data_with_alps = pd.read_pickle("/Users/nathansuberi/Desktop/RW_Data/wfp_alps.pkl")
df = wfp_data_with_alps

In [14]:
df[df["Model Std Error of Residuals"] > 10000]

,adm0_id,adm0_name,adm1_id,adm1_name,mkt_id,mkt_name,cm_id,cm_name,cur_id,cur_name,...,um_name,mp_month,mp_year,mp_price,mp_commoditysource,ALPS,Fitted Model,Fitted Model Tame,Model Residual,Model Std Error of Residuals
405,1,Afghanistan,272,Badakhshan,266,Fayzabad,284,Fuel (diesel),87,AFN,...,L,3,2000,50000.0000,WFP,yellow,"LinearRegression(copy_X=True, fit_intercept=Tr...","LinearRegression(copy_X=True, fit_intercept=Tr...",0.322496,24508.413235
406,1,Afghanistan,272,Badakhshan,266,Fayzabad,284,Fuel (diesel),87,AFN,...,L,4,2000,49000.0000,WFP,yellow,"LinearRegression(copy_X=True, fit_intercept=Tr...","LinearRegression(copy_X=True, fit_intercept=Tr...",0.372125,24508.413235
407,1,Afghanistan,272,Badakhshan,266,Fayzabad,284,Fuel (diesel),87,AFN,...,L,5,2000,44625.0000,WFP,white,"LinearRegression(copy_X=True, fit_intercept=Tr...","LinearRegression(copy_X=True, fit_intercept=Tr...",0.221353,24508.413235
408,1,Afghanistan,272,Badakhshan,266,Fayzabad,284,Fuel (diesel),87,AFN,...,L,6,2000,59800.0000,WFP,yellow,"LinearRegression(copy_X=True, fit_intercept=Tr...","LinearRegression(copy_X=True, fit_intercept=Tr...",0.891333,24508.413235
409,1,Afghanistan,272,Badakhshan,266,Fayzabad,284,Fuel (diesel),87,AFN,...,L,7,2000,62500.0000,WFP,orange,"LinearRegression(copy_X=True, fit_intercept=Tr...","LinearRegression(copy_X=True, fit_intercept=Tr...",1.018077,24508.413235
410,1,Afghanistan,272,Badakhshan,266,Fayzabad,284,Fuel (diesel),87,AFN,...,L,8,2000,53000.0000,WFP,yellow,"LinearRegression(copy_X=True, fit_intercept=Tr...","LinearRegression(copy_X=True, fit_intercept=Tr...",0.655143,24508.413235
411,1,Afghanistan,272,Badakhshan,266,Fayzabad,284,Fuel (diesel),87,AFN,...,L,9,2000,59583.2500,WFP,yellow,"LinearRegression(copy_X=True, fit_intercept=Tr...","LinearRegression(copy_X=True, fit_intercept=Tr...",0.887291,24508.413235
412,1,Afghanistan,272,Badakhshan,266,Fayzabad,284,Fuel (diesel),87,AFN,...,L,10,2000,85657.7500,WFP,orange,"LinearRegression(copy_X=True, fit_intercept=Tr...","LinearRegression(copy_X=True, fit_intercept=Tr...",1.812856,24508.413235
413,1,Afghanistan,272,Badakhshan,266,Fayzabad,284,Fuel (diesel),87,AFN,...,L,11,2000,147263.0000,WFP,red,"LinearRegression(copy_X=True, fit_intercept=Tr...","LinearRegression(copy_X=True, fit_intercept=Tr...",3.950519,24508.413235
414,1,Afghanistan,272,Badakhshan,266,Fayzabad,284,Fuel (diesel),87,AFN,...,L,12,2000,109027.2500,WFP,red,"LinearRegression(copy_X=True, fit_intercept=Tr...","LinearRegression(copy_X=True, fit_intercept=Tr...",2.674155,24508.413235


In [ ]:
df

In [9]:
adm1_names = df["adm1_name"].unique()

In [15]:
### Matching with GADM files ###

import requests as req

sql = "SELECT * FROM gadm28_adm1"
limit = 9000
sql = sql.format(limit)

rw_id = "098b33df-6871-4e53-a5ff-b56a7d989f9a"
query_base = "https://api.resourcewatch.org/v1/query/{}?sql={}"
query2 = query_base.format(rw_id, sql)

payload = { "application":"rw", "page[size]": 10000000}
res2 = req.get(query2, params=payload)

In [4]:
import fiona
# import geopandas as gpd
# Use geopandas or fiona to merge ALPS data with a shapefile - try GADM

In [12]:
file = "/Users/nathansuberi/Desktop/RW_Data/gadm28/gadm28.shp"
adm1_features = {}
with fiona.open(file) as src:
    print(len(src))
    for i, feature in enumerate(src):
        if i%10000==0:
            print(i)
        name = feature["properties"]["NAME_1"] 
        if(name in adm1_names):
            if(name not in adm1_features):
                adm1_features[name] = feature["geometry"]

255272
0
10000
20000
30000
40000
50000
60000
70000
80000
90000
100000
110000
120000
130000
140000
150000
160000
170000
180000
190000
200000
210000
220000
230000
240000
250000


In [ ]:
adm1_features

In [17]:
group_df = df.groupby(["adm1_name", "ALPS", "mp_year"])
#group_df.groups
#group_df.groups.keys()
#group_df.get_group('#Antananarivo')

#group_df.groups

In [21]:
pd.options.display.max_rows = 10000
group_df.count().pivot_table(values="adm0_id", index=["mp_year", "adm1_name"], columns=["ALPS"])

ALPS                                                 orange    red  white  \
mp_year adm1_name                                                           
1992    Boucle Du Mouhoun                               NaN    NaN   11.0   
        Cascades                                        NaN    NaN    9.0   
        Centre                                          NaN    NaN   10.0   
        Centre-est                                      NaN    NaN   10.0   
        Centre-nord                                     NaN    NaN   12.0   
        Centre-ouest                                    NaN    NaN   12.0   
        Est                                             NaN    NaN    6.0   
        Sahel                                           NaN    NaN   33.0   
        Sud-ouest                                       NaN    NaN   12.0   
1993    Boucle Du Mouhoun                               NaN    NaN   12.0   
        Cascades                                        NaN    NaN   12.0   
        Centre                                          NaN    NaN   12.0   
        Centre-est                                      NaN    NaN   12.0   
        Centre-nord                                     NaN    NaN   12.0   
        Centre-ouest                                    NaN    NaN   12.0   
        Est                                             NaN    NaN   12.0   
        Sahel                                           NaN    NaN   36.0   
        Sud-ouest                                       NaN    NaN   12.0   
1994    Agadez                                          2.0    NaN   52.0   
        Andhra Pradesh                                 45.0    NaN    NaN   
        Assam                                          13.0    NaN    NaN   
        Bihar                                           3.0    1.0    NaN   
        Boucle Du Mouhoun                               NaN    NaN   10.0   
        Cascades                                        NaN    NaN   12.0   
        Centre                                          NaN    NaN   12.0   
        Centre-est                                      NaN    NaN   12.0   
        Centre-nord                                     NaN    NaN   12.0   
        Centre-ouest                                    NaN    NaN   12.0   
        Delhi                                          47.0    NaN    NaN   
        Diffa                                           2.0    NaN   45.0   
        Dosso                                           1.0    NaN   72.0   
        Est                                             NaN    NaN   12.0   
        Gujarat                                        30.0    2.0    NaN   
        Himachal Pradesh                               33.0    NaN    NaN   
        Karnataka                                      15.0    1.0    1.0   
        Kerala                                         13.0    NaN    NaN   
        Madhya Pradesh                                 28.0    NaN    NaN   
        Maharashtra                                    18.0    1.0   11.0   
        Maradi                                          1.0    NaN   72.0   
        Meghalaya                                      13.0    NaN    NaN   
        Mizoram                                        11.0    NaN    NaN   
        Niamey                                          NaN    NaN   45.0   
        Orissa                                         24.0    NaN    1.0   
        Rajasthan                                      31.0    1.0    NaN   
        Sahel                                           NaN    NaN   36.0   
        Sud-ouest                                       NaN    NaN   12.0   
        Tahoua                                          3.0    NaN   54.0   
        Tamil Nadu                                     20.0    NaN    1.0   
        Tillaberi                                       7.0    NaN  101.0   
        Tripura                                        17.0    NaN    NaN

In [35]:
import numpy as np
df = pd.DataFrame({'A' : ['foo', 'bar', 'foo', 'bar',
                           'foo', 'bar', 'foo', 'foo'],
                   'B' : ['one', 'one', 'two', 'three',
                          'two', 'two', 'one', 'three'],
                   'C' : np.random.randn(8),
                   'D' : np.random.randn(8)})

grouped = df.groupby(['A', 'B'])
grouped.groups

{('bar', 'one'): Int64Index([1], dtype='int64'),
 ('bar', 'three'): Int64Index([3], dtype='int64'),
 ('bar', 'two'): Int64Index([5], dtype='int64'),
 ('foo', 'one'): Int64Index([0, 6], dtype='int64'),
 ('foo', 'three'): Int64Index([7], dtype='int64'),
 ('foo', 'two'): Int64Index([2, 4], dtype='int64')}

In [26]:
red_alerts_fayzabad = df.loc[(df["ALPS"]=="red") & (df["mkt_name"]=="Fayzabad") , ["adm0_name", "adm1_name", "mkt_name", "cm_name", "cur_name", "um_name", "mp_month", "mp_year", "mp_price"]]
print(red_alerts.head(10))

red_alerts_fayzabad.to_csv("/Users/nathansuberi/Desktop/RW_Data/wfp_red_alerts_fayzabad.csv")

       adm0_name   adm1_name  mkt_name             cm_name cur_name um_name  \
103  Afghanistan  Badakhshan  Fayzabad               Wheat      AFN      KG   
104  Afghanistan  Badakhshan  Fayzabad               Wheat      AFN      KG   
105  Afghanistan  Badakhshan  Fayzabad               Wheat      AFN      KG   
106  Afghanistan  Badakhshan  Fayzabad               Wheat      AFN      KG   
107  Afghanistan  Badakhshan  Fayzabad               Wheat      AFN      KG   
108  Afghanistan  Badakhshan  Fayzabad               Wheat      AFN      KG   
109  Afghanistan  Badakhshan  Fayzabad               Wheat      AFN      KG   
110  Afghanistan  Badakhshan  Fayzabad               Wheat      AFN      KG   
232  Afghanistan  Badakhshan  Fayzabad  Rice (low quality)      AFN      KG   
233  Afghanistan  Badakhshan  Fayzabad  Rice (low quality)      AFN      KG   

     mp_month  mp_year  mp_price  
103         4     2008    37.275  
104         5     2008    44.400  
105         6     2008   

In [22]:
not_represented = [key for key in adm1_names if key not in adm1_features.keys()]
print(len(not_represented)/len(adm1_names))

0.4533106960950764


In [ ]:
import rasterio as rio

# Use rasterio to rasterize the vector, print to a geotiff

In [35]:
red_alerts_fayzabad = df.loc[(df["ALPS"]=="red") & (df["mkt_name"]=="Fayzabad") , ["adm0_name", "adm1_name", "mkt_name", "cm_name", "cur_name", "um_name", "mp_month", "mp_year", "mp_price"]]
red_alerts_fayzabad.head(10)

,adm0_name,adm1_name,mkt_name,cm_name,cur_name,um_name,mp_month,mp_year,mp_price
103,Afghanistan,Badakhshan,Fayzabad,Wheat,AFN,KG,4,2008,37.275
104,Afghanistan,Badakhshan,Fayzabad,Wheat,AFN,KG,5,2008,44.400
105,Afghanistan,Badakhshan,Fayzabad,Wheat,AFN,KG,6,2008,33.000
106,Afghanistan,Badakhshan,Fayzabad,Wheat,AFN,KG,7,2008,34.775
107,Afghanistan,Badakhshan,Fayzabad,Wheat,AFN,KG,8,2008,34.950
108,Afghanistan,Badakhshan,Fayzabad,Wheat,AFN,KG,9,2008,35.700
109,Afghanistan,Badakhshan,Fayzabad,Wheat,AFN,KG,10,2008,32.300
110,Afghanistan,Badakhshan,Fayzabad,Wheat,AFN,KG,11,2008,27.975
232,Afghanistan,Badakhshan,Fayzabad,Rice (low quality),AFN,KG,9,2008,64.000
233,Afghanistan,Badakhshan,Fayzabad,Rice (low quality),AFN,KG,10,2008,67.750


In [34]:
tuples = list(zip(red_alerts_fayzabad["mp_year"], red_alerts_fayzabad["mp_month"]))
multi_index = pd.MultiIndex.from_tuples(tuples, names=["Year", "Month"])
red_alerts_fayzabad.index = multi_index
red_alerts_fayzabad.loc[(2008, 5)]

/Users/nathansuberi/.pyenv/versions/3.6.0/lib/python3.6/site-packages/ipykernel_launcher.py:4: PerformanceWarning: indexing past lexsort depth may impact performance.
  after removing the cwd from sys.path.


,,adm0_name,adm1_name,mkt_name,cm_name,cur_name,um_name,mp_month,mp_year,mp_price
Year,Month,,,,,,,,,
2008,5,Afghanistan,Badakhshan,Fayzabad,Wheat,AFN,KG,5,2008,44.4


In [40]:
multi_index = pd.MultiIndex.from_arrays([red_alerts_fayzabad["mp_year"], red_alerts_fayzabad["mp_month"]], names=["Year", "Month"])
red_alerts_fayzabad.index = multi_index
red_alerts_fayzabad


adm0_name   adm1_name  mkt_name  \
Year Month                                      
2008 4      Afghanistan  Badakhshan  Fayzabad   
     5      Afghanistan  Badakhshan  Fayzabad   
     6      Afghanistan  Badakhshan  Fayzabad   
     7      Afghanistan  Badakhshan  Fayzabad   
     8      Afghanistan  Badakhshan  Fayzabad   
     9      Afghanistan  Badakhshan  Fayzabad   
     10     Afghanistan  Badakhshan  Fayzabad   
     11     Afghanistan  Badakhshan  Fayzabad   
     9      Afghanistan  Badakhshan  Fayzabad   
     10     Afghanistan  Badakhshan  Fayzabad   
     11     Afghanistan  Badakhshan  Fayzabad   
     12     Afghanistan  Badakhshan  Fayzabad   
2009 1      Afghanistan  Badakhshan  Fayzabad   
2000 11     Afghanistan  Badakhshan  Fayzabad   
     12     Afghanistan  Badakhshan  Fayzabad   
2001 1      Afghanistan  Badakhshan  Fayzabad   
     3      Afghanistan  Badakhshan  Fayzabad   
2002 10     Afghanistan  Badakhshan  Fayzabad   
     11     Afghanistan  Badakhshan  Fayzabad   
     12     Afghanistan  Badakhshan  Fayzabad   

                                                  cm_name cur_name um_name  \
Year Month                                                                   
2008 4                                              Wheat      AFN      KG   
     5                                              Wheat      AFN      KG   
     6                                              Wheat      AFN      KG   
     7                                              Wheat      AFN      KG   
     8                                              Wheat      AFN      KG   
     9                                              Wheat      AFN      KG   
     10                                             Wheat      AFN      KG   
     11                                             Wheat      AFN      KG   
     9                                 Rice (low quality)      AFN      KG   
     10                                Rice (low quality)      AFN      KG   
     11                                Rice (low quality)      AFN      KG   
     12                                Rice (low quality)      AFN      KG   
2009 1                                 Rice (low quality)      AFN      KG   
2000 11                                     Fuel (diesel)      AFN       L   
     12                                     Fuel (diesel)      AFN       L   
2001 1                                      Fuel (diesel)      AFN       L   
     3                                      Fuel (diesel)      AFN       L   
2002 10     Wage (non-qualified labour, non-agricultural)      AFN     Day   
     11     Wage (non-qualified labour, non-agricultural)      AFN     Day   
     12     Wage (non-qualified labour, non-agricultural)      AFN     Day   

            mp_month  mp_year    mp_price  
Year Month                                 
2008 4             4     2008      37.275  
     5             5     2008      44.400  
     6             6     2008      33.000  
     7             7     2008      34.775  
     8             8     2008      34.950  
     9             9     2008      35.700  
     10           10     2008      32.300  
     11           11     2008      27.975  
     9             9     2008      64.000  
     10           10     2008      67.750  
     11           11     2008      64.000  
     12           12     2008      64.000  
2009 1             1     2009      68.500  
2000 11           11     2000  147263.000  
     12           12     2000  109027.250  
2001 1             1     2001   93500.000  
     3             3     2001   97500.000  
2002 10           10     2002  185000.000  
     11           11     2002  180000.000  
     12           12     2002  200000.000